## Скрипт для создания заявок Subzero на несколько аккаунтов МС


In [1]:
# ! pip install psycopg2-binary
# ! pip install snowflake-connector-python
# !pip install ipython-sql
import time
import pandas as pd
import requests
import json
from concurrent.futures import ThreadPoolExecutor
import sqlalchemy

con = sqlalchemy.create_engine('postgresql://postgres:@127.0.0.1:8181/lognex')

In [2]:
# Задание переменных:
namespace = 5  # Неймспейс
ticket_number = 1  # Количество заявок на 1 аккаунт
thread_number = 20  # Количество потоков

url = f'https://subzero-billing-{namespace}.testms-test.lognex.ru/api/clinton/1.0/ticket'
headers = {'Content-Type': 'application/json'}

In [12]:
# Фукнция для запроса к базе

def sql_go(request):
    return pd.read_sql(sqlalchemy.text(request), con)

In [4]:
# Вручную выгруженный файл из базы с аккаунтами
# accounts = pd.read_csv('lognex_billing_billingaccount.csv',header=None)
# accounts=accounts.values.tolist()

In [13]:
# Запрос
sql = '''
            select id from billing.billingaccount WHERE company LIKE 'sobaka%'
        '''

In [14]:
accounts = sql_go(sql)
# print(accounts)
# print(accounts.info())

## Функция для скрипта

In [15]:
def bomber_to_many():
    counter = 0
    with open('json3.json', 'r') as json3:
        sub_data = json.load(json3)
    for index, row in accounts.iterrows():
        sub_data['accountId'] = str(row["id"])

        for i in range(ticket_number):
            counter += 1
            response = requests.post(url, data=json.dumps(sub_data), headers=headers)
            if response.ok:
                print(f'Заявка для {row} создана', '\n')
            #return response
            else:
                print(f'Заявка для {row} НЕ создана'+str(response))
    print('------------------------------------------------')
    print(f'итого, создано заявок: {counter}')


## Одиночный прогон без потоков

In [ ]:
bomber_to_many

## Потоки

### Пуск потоков

In [ ]:


start = time.time()  ## точка отсчета времени
with ThreadPoolExecutor(max_workers=thread_number) as executor:
    # Запуск потоков
    results = [executor.submit(bomber_to_many) for _ in range(thread_number)]

end = time.time() - start
print('All threads have completed')
print(f'Выполнялось: {end}с' )  ## вывод времени